In [10]:
# ライブラリimport

from socket import socket, AF_INET, SOCK_DGRAM
from struct import *
from datetime import *

def print_bin(label, bin):
	print(label, end=" : ")
	for i in bin:
		print(format(i, "02x"), end=" ")
	print("")

# UDPソケット

SOCKET_BUFFSIZE = 2048
ETHIO_SERVER = ('192.168.1.203', 16241)
sock = socket(AF_INET, SOCK_DGRAM)

# パケット作成

ETHIO_PACKET_FOURCC = b'AVMM'
ETHIO_PACKET_ENDCMD = b'\x7f\x00\xff\xff'

def data_padding(data):
	return data + b'\x00' * ((4 - len(data)) & 3)

def cmd_memwr(addr, wrbyte):
	return pack('!BxHL', 0x44, len(wrbyte), addr) + data_padding(wrbyte)

def cmd_memrd(addr, readnum):
	return pack('!BxHL', 0x54, readnum, addr)

def cmd_fifowr(ch, wrbyte):
	return pack('!BxH', 0x20 | (ch & 15), len(wrbyte)) + data_padding(wrbyte)

def cmd_fiford(ch, readnum):
	return pack('!BxH', 0x30 | (ch & 15), readnum)


In [12]:
# Avalon-MMリクエストパケット

send_bin = ETHIO_PACKET_FOURCC + cmd_memrd(0x10000000, 8) + ETHIO_PACKET_ENDCMD
print_bin("send packet", send_bin)

# UDP送受信 

sock.sendto(send_bin, ETHIO_SERVER)
res_bin, addr = sock.recvfrom(SOCKET_BUFFSIZE)

# レスポンスパケット

print_bin("recv packet",res_bin)

# PERIDOTのSystem IDペリフェラルの値を取り出す

sysid, timestamp = unpack('LL', res_bin[8:16])
dt = datetime.fromtimestamp(timestamp)
print("System ID =",hex(sysid),", Build",dt)


send packet : 41 56 4d 4d 54 00 00 08 10 00 00 00 7f 00 ff ff 
recv packet : 41 56 4d 4d d4 00 00 08 01 00 c0 72 f8 56 34 63 ff 00 00 00 
System ID = 0x72c00001 , Build 2022-09-28 23:15:20


In [14]:
# FIFO書き込み＆FIFO読み出しのリクエスト

fifo_data = b'\xf0' + bytes(range(0x01, 0x100)) + b'\xf1' + bytes(range(0x01, 0xff))
padding = b'\x00' * ((4 - len(fifo_data)) & 3)

send_bin = ETHIO_PACKET_FOURCC
send_bin += cmd_fifowr(0, fifo_data)
send_bin += cmd_fiford(0, 1024)
send_bin += ETHIO_PACKET_ENDCMD
print_bin("send packet", send_bin)

# UDP送受信

sock.sendto(send_bin, ETHIO_SERVER)
res_bin, addr = sock.recvfrom(SOCKET_BUFFSIZE)

print_bin("recv packet",res_bin)


send packet : 41 56 4d 4d 20 00 01 ff f0 01 02 03 04 05 06 07 08 09 0a 0b 0c 0d 0e 0f 10 11 12 13 14 15 16 17 18 19 1a 1b 1c 1d 1e 1f 20 21 22 23 24 25 26 27 28 29 2a 2b 2c 2d 2e 2f 30 31 32 33 34 35 36 37 38 39 3a 3b 3c 3d 3e 3f 40 41 42 43 44 45 46 47 48 49 4a 4b 4c 4d 4e 4f 50 51 52 53 54 55 56 57 58 59 5a 5b 5c 5d 5e 5f 60 61 62 63 64 65 66 67 68 69 6a 6b 6c 6d 6e 6f 70 71 72 73 74 75 76 77 78 79 7a 7b 7c 7d 7e 7f 80 81 82 83 84 85 86 87 88 89 8a 8b 8c 8d 8e 8f 90 91 92 93 94 95 96 97 98 99 9a 9b 9c 9d 9e 9f a0 a1 a2 a3 a4 a5 a6 a7 a8 a9 aa ab ac ad ae af b0 b1 b2 b3 b4 b5 b6 b7 b8 b9 ba bb bc bd be bf c0 c1 c2 c3 c4 c5 c6 c7 c8 c9 ca cb cc cd ce cf d0 d1 d2 d3 d4 d5 d6 d7 d8 d9 da db dc dd de df e0 e1 e2 e3 e4 e5 e6 e7 e8 e9 ea eb ec ed ee ef f0 f1 f2 f3 f4 f5 f6 f7 f8 f9 fa fb fc fd fe ff f1 01 02 03 04 05 06 07 08 09 0a 0b 0c 0d 0e 0f 10 11 12 13 14 15 16 17 18 19 1a 1b 1c 1d 1e 1f 20 21 22 23 24 25 26 27 28 29 2a 2b 2c 2d 2e 2f 30 31 32 33 34 35 36 37 38 39 3a 3b 3c 3d 3e 3f 40

In [34]:
# WSLEDのテスト

def pixel(r, g, b):
	r = r if r<254 else 254
	g = g if g<254 else 254
	b = b if b<254 else 254
	return pack('BBB', g,r,b)

#led_bin = (pixel(255,0,0) + pixel(0,255,0) + pixel(0,0,255)) * 12 + b'\xff'
led_bin = pixel(0,0,0) * 37

send_bin = ETHIO_PACKET_FOURCC + cmd_fifowr(1, led_bin) + ETHIO_PACKET_ENDCMD
print_bin("send packet", send_bin)

# UDP送受信

sock.sendto(send_bin, ETHIO_SERVER)
res_bin, addr = sock.recvfrom(SOCKET_BUFFSIZE)

print_bin("recv packet",res_bin)


send packet : 41 56 4d 4d 21 00 00 6f 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 7f 00 ff ff 
recv packet : 41 56 4d 4d a1 00 00 6f ff 00 00 00 
